# Plotting the flights network on a US map

*Mapping the network for each of the years 1998, 2007, and 2018.*

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point
# import copy

Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.


## Load data 

Airports data comes from https://openflights.org/data.html.

Flights data comes from the US Bureau of Transportation Statistics (BTS), ([source](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236)).

See our [data compiling](https://github.com/Morten-Esketveit/TSDS-gruppe-2019/blob/master/Exam/Data_compiling_v3.ipynb) file.

In [3]:
# Load airports data, 2007
Airports = pd.read_pickle("Data/Airports.pkl").reset_index(drop=True)
Airports.tail(2)

,Airport_id,Name,City,Country,3DigitId,Lat,Lon,Altitude_ft
1510,13759,Oswego County Airport,Fulton,United States,\N,43.350800,-76.388100,475
1511,13803,Mitchell Municipal Airport,Mitchell,United States,MHE,43.774799,-98.038597,1304


In [4]:
# Load flights data, 1998
FlightsNx98 = pd.read_pickle("Data/FlightsNx98.pkl")
FlightsNx98.tail(2)

,Origin,Dest,Distance,count,avg_time_mins,companies,Origin_flights,Destination_flights,origin_degree,dest_degree,origin_btwns,dest_btwns,origin_clustcoef,dest_clustcoef
3155,EGE,SFO,847.0,9,154.125,1,2559.0,279737.0,12.0,51.0,0.000030,0.049559,0.969697,0.465098
3156,CLE,SRQ,967.0,17,155.000,1,122145.0,11971.0,42.0,10.0,0.003055,0.000005,0.706156,0.977778


In [5]:
# Load flights data, 2007
FlightsNx07 = pd.read_pickle("Data/FlightsNx07.pkl")
FlightsNx07.tail(2)

,Origin,Dest,Distance,count,avg_time_mins,companies,Origin_flights,Destination_flights,origin_degree,dest_degree,origin_btwns,dest_btwns,origin_clustcoef,dest_clustcoef
4945,SYR,IAD,296.0,372,77.475410,1,27025,189940.0,14.0,66.0,0.000002,0.006780,0.967033,0.458275
4946,YUM,LAS,238.0,78,75.692308,1,6928,371600.0,4.0,87.0,0.000097,0.019032,0.666667,0.365678


In [6]:
# Load flights data, 2018
FlightsNx18 = pd.read_pickle("Data/FlightsNx18.pkl")
FlightsNx18.tail(2)

,Origin,Dest,Distance,count,avg_time_mins,companies,Origin_flights,Destination_flights,origin_degree,dest_degree,origin_btwns,dest_btwns,origin_clustcoef,dest_clustcoef
6223,LGA,SBN,651.0,3,195.000000,1,339632,13894,73.0,13.0,0.006823,0.000534,0.350457,0.461538
6224,ORD,ABE,654.0,225,118.458333,2,737786,8295,172.0,10.0,0.141068,0.000229,0.138923,0.511111


## Implement as network. 
For function documentation, see:  
https://networkx.github.io/documentation/stable/reference/generated/networkx.convert_matrix.from_pandas_edgelist.html?highlight=from_pandas_edgelist#networkx.convert_matrix.from_pandas_edgelist

In [31]:
class US_graph(object):
    ''' Creates a map network of flights for the US.
    '''
    
    def __init__(self, flights, airports, scalar=6, alpha=0.5):
        """ Read the data and create the network
            Optional: Set size-scalar and alpha for the nodes.
        """
        self.f = flights # Dataset on flights
        self.air = airports
        self.s = scalar # scalar for the size of the nodes
        self.a = alpha
        self.G = nx.from_pandas_edgelist(self.f, source = "Origin", target = "Dest", edge_attr = True)
        print(nx.info(self.G))


    def setup(self):
        """ Setup nodes, attributes and positions for drawing
        """ 
        # Get list of nodes, and add lat, lon from Airports dataset:
        nodes = pd.DataFrame(list(self.G.nodes()))
        nodes.columns = ["3DigitId"]
        positions = nodes.merge(self.air[["3DigitId","Lat","Lon"]], how = "left", on = "3DigitId")

        # Create a positions dictionary, which is in the format necessary to add it as a node attribute.
        pos_= dict()
        for idx, row in positions.iterrows():
            pos_[row['3DigitId']] = dict(pos=(row['Lat'], row['Lon']))

        # Set the node attributes. 
        nx.set_node_attributes(self.G, pos_)
        attributes = positions.merge(self.f[["Origin","Origin_flights","origin_degree"]], how = "left", left_on = "3DigitId", right_on = "Origin").drop_duplicates()
        attributes = attributes.sort_values(by = "origin_degree", ascending = False)

        # Positions for drawing
        pos = {city:(long, lat) for (city, (lat,long)) in nx.get_node_attributes(self.G, 'pos').items()}

        return nodes, attributes, pos


    def hubs(self):
        """ Setup hubs and position dictionary for labels
        """ 
        nodes, attributes, pos = self.setup()
        # Split airports according to size/importance for plotting purposes:
        airports = list(attributes.loc[attributes["origin_degree"]<=25]["Origin"])
        hubs_small = list(attributes.loc[attributes["origin_degree"]>25]["Origin"])
        hubs = list(attributes.loc[attributes["origin_degree"]>50]["Origin"])
        hubs_large = list(attributes.loc[attributes["origin_degree"]>100]["Origin"])

        # label location: how far above the center of the node
        raise_by = .8 + self.s*.1
        # Label large hubs for draw_networkx_labels
        pos_large = dict((k, (pos[k][0], pos[k][1]+raise_by)) for k in hubs_large if k in pos)
        # Label medium hubs for draw_networkx_labels (in case of no large hubs)
        pos_med = dict((k, (pos[k][0], pos[k][1]+raise_by)) for k in hubs if k in pos)

        return airports, hubs_small, hubs, hubs_large, pos_large, pos_med

    
    def degree(self, subset):
        """ Function for degree size
            Returning size = degree*scalar for a given threshold
        """
        nodes, attributes, pos = self.setup()
        attributes['origin_degree'].fillna(0, inplace=True)

        return [int(attributes.loc[attributes["3DigitId"]==i]['origin_degree'])*self.s for i in subset]

    
    def draw(self, path):
        """ Draw the map with airports, small hubs, hubs, and large hubs
            The options are given by the __init__ above
        """
        nodes, attributes, pos = self.setup()
        airports, hubs_small, hubs, hubs_large, pos_large, pos_med = self.hubs()
        
        # Get geographical file from GeoPandas
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
        # Select US: 
        US = world[(world.pop_est>0) & (world.name == "United States")]

        fig, ax = plt.subplots(figsize = (16,12))
        ax.set_facecolor("lightblue")
        world.plot(ax = ax, color = "lightgrey", edgecolor = "black")
        US.plot(ax = ax, color = "whitesmoke", edgecolor = "black")
        plt.xlim(-126,-66)
        plt.ylim(24,50)
        nx.draw_networkx_edges(self.G, pos, width=0.05, with_labels=False, ax = ax)
        nx.draw_networkx_nodes(self.G, pos, nodelist = airports, alpha=self.a, node_size=self.degree(airports), node_color = "blue", with_labels=False, ax = ax).set_label("Airport")
        nx.draw_networkx_nodes(self.G, pos, nodelist = hubs_small, alpha=self.a, node_size=self.degree(hubs_small), node_color = "purple", with_labels=False, ax = ax).set_label("Small hub, degree > 25")
        nx.draw_networkx_nodes(self.G, pos, nodelist = hubs, alpha=self.a, node_size=self.degree(hubs), node_color = "yellow", with_labels=False, ax = ax).set_label("Medium hub, degree > 50")
        if hubs_large != []:
            nx.draw_networkx_nodes(self.G, pos, nodelist = hubs_large, alpha=self.a, node_size=self.degree(hubs_large), node_color = "red", with_labels=False, ax = ax).set_label("Large hub, degree > 100")
            nx.draw_networkx_labels(self.G.subgraph(hubs_large), pos_large, font_color='k', font_size=14)
        else:
            nx.draw_networkx_labels(self.G.subgraph(hubs), pos_med, font_color='k', font_size=14)
        plt.xticks([])
        plt.yticks([])
        plt.legend(fontsize = 14, loc='lower left')
        fig.savefig(path, bbox_inches='tight')
        plt.show()

    def ego(self, single_airport, path):
        """ Draw the connections for a single airport
            Using same map with airports, small hubs, hubs, and large hubs
            The options are given by the __init__ above
        """
        nodes, attributes, pos = self.setup()
        airports, hubs_small, hubs, hubs_large, pos_large, pos_med = self.hubs()

        # Setup the ego-graph containing one single airport and it's neighbors
        ego = nx.ego_graph(self.G, single_airport)
        neighbors = list(nx.all_neighbors(ego, single_airport))
        neighbors = neighbors.append(single_airport)
        airports = list(set(airports).intersection(neighbors))
        hubs_small = list(set(hubs_small).intersection(neighbors))
        hubs = list(set(hubs).intersection(neighbors))
        hubs_large = list(set(hubs_large).intersection(ego_neighbors))
        
        # Get geographical file from GeoPandas
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
        # Select US: 
        US = world[(world.pop_est>0) & (world.name == "United States")]

        fig, ax = plt.subplots(figsize = (16,12))
        ax.set_facecolor("lightblue")
        world.plot(ax = ax, color = "lightgrey", edgecolor = "black")
        US.plot(ax = ax, color = "whitesmoke", edgecolor = "black")
        plt.xlim(-126,-66)
        plt.ylim(24,50)
        nx.draw_networkx_edges(ego, pos, width=0.05, with_labels=False, ax = ax)
        nx.draw_networkx_nodes(ego, pos, nodelist = airports, alpha=self.a, node_size=self.degree(airports), node_color = "blue", with_labels=False, ax = ax).set_label("Airport")
        nx.draw_networkx_nodes(ego, pos, nodelist = hubs_small, alpha=self.a, node_size=self.degree(hubs_small), node_color = "purple", with_labels=False, ax = ax).set_label("Small hub, degree > 25")
        nx.draw_networkx_nodes(ego, pos, nodelist = hubs, alpha=self.a, node_size=self.degree(hubs), node_color = "yellow", with_labels=False, ax = ax).set_label("Medium hub, degree > 50")
        if hubs_large != []:
            nx.draw_networkx_nodes(self.G, pos, nodelist = hubs_large, alpha=self.a, node_size=self.degree(hubs_large), node_color = "red", with_labels=False, ax = ax).set_label("Large hub, degree > 100")
            nx.draw_networkx_labels(self.G.subgraph(hubs_large), pos_large, font_color='k', font_size=14)
        else:
            nx.draw_networkx_labels(self.G.subgraph(hubs), pos_med, font_color='k', font_size=14)
        plt.xticks([])
        plt.yticks([])
        plt.legend(fontsize = 14, loc='lower left')
        fig.savefig(path, bbox_inches='tight')
        plt.show()


## Draw general maps

In [30]:
single_airport = 'LAX'
neighbors = ['PHL', 'PDX']
neighbors.append(single_airport)
print(neighbors)

['PHL', 'PDX', 'LAX']


### 1998

In [32]:
G98 = US_graph(FlightsNx98, Airports)
G98.ego('LAX', path='Figures/map_LAX_98.png')
G98.draw(path='Figures/map_general_98.png')

Name: 
Type: Graph
Number of nodes: 209
Number of edges: 1614
Average degree:  15.4450


TypeError: 'NoneType' object is not iterable

### 2007

In [ ]:
G07 = US_graph(FlightsNx07, Airports)

G07.draw(path='Figures/map_general_07.png')

### 2018

In [ ]:
G18 = US_graph(FlightsNx18, Airports)

G18.draw(path='Figures/map_general_18.png')

## Draw ego maps

For Los Angeles International Airport and it's neighbors

In [ ]:
# 1998
G98.ego('LAX', path='Figures/map_LAX_98.png')